# 1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender

In [68]:
import pandas as pd
import numpy as np
import dask.dataframe as ddfrom 
from IPython.display import display

In [2]:
loans_lenders=dd.read_csv('/Users/caterinapisani/Progetto FCS/KIVA/loans_lenders.csv')

#### Prima di risolvere il problema posto, voglio avere una breve overview del dataset. Dunque, come prima cosa, stampo la "testa" (prime 5 righe) e la "coda" (ultime 5 righe) del dataset e eseguo una count del numero totale di righe.

In [3]:
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [4]:
loans_lenders.tail()

,loan_id,lenders
138359,678999,"michael43411218, carol5987, gooddogg1, chris41..."
138360,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
138361,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
138362,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."
138363,1206486,"alan5175, amy38101311"


In [5]:
len(loans_lenders)

1387432

#### Successivamente, voglio sapere se e quanti valori NA o duplicati sono presenti nella colonna "loan_id". 

In [6]:
loans_lenders.isna().sum().compute()

loan_id    0
lenders    0
dtype: int64

In [7]:
len(loans_lenders['loan_id'].unique().compute())

1387432

#### Noto che non ci sono nè valori nulli nè duplicati nella colonna "loan_id".

#### Ora posso procedere con la normalizzazione della tabella. Siccome le stringhe della colonna "lenders" sono separate da virgola, eseguo prima di tutto  una split. Dopodichè posso applicare la funzione "explode", che letteralmente fa "esplodere" le stringhe su più righe, così da avere la tabella in forma normale. 

In [3]:
norm_loans_lenders = loans_lenders.assign(lenders=loans_lenders["lenders"].str.split(",")).explode("lenders").reset_index(drop=True)

In [9]:
norm_loans_lenders.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [10]:
norm_loans_lenders.columns

Index(['loan_id', 'lenders'], dtype='object')

In [11]:
len(norm_loans_lenders)

28293931

In [12]:
len(norm_loans_lenders['lenders'].unique().compute())

1639026

# 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [4]:
#Leggo il file e includo un'opzione per poter visualizzare tutte le colonne 
loans=dd.read_csv('/Users/caterinapisani/Progetto FCS/KIVA/loans.csv')
pd.set_option('display.max_columns', None)

In [14]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [15]:
#Voglio sapere il tipo di dati della tabella, ed in particolare noto che "planned_expiration_time"
#e "disburse_time" sono considerati oggetti, non sono ancora in formato datetime.
loans.dtypes

loan_id                              int64
loan_name                           object
original_language                   object
description                         object
description_translated              object
funded_amount                      float64
loan_amount                        float64
status                              object
activity_name                       object
sector_name                         object
loan_use                            object
country_code                        object
country_name                        object
town_name                           object
currency_policy                     object
currency_exchange_coverage_rate    float64
currency                            object
partner_id                         float64
posted_time                         object
planned_expiration_time             object
disburse_time                       object
raised_time                         object
lender_term                        float64
num_lenders

In [16]:
#Conto i valori mancanti nella colonna "planned_expiration_time"
Na_planned = loans.planned_expiration_time.isna().sum().compute()
Na_planned

371834

In [17]:
#Conto i valori mancanti nella colonna "disburse_time"
Na_disburse = loans.disburse_time.isna().sum().compute()
Na_disburse

2813

In [18]:
Tot_Na= Na_planned + Na_disburse
Tot_Na

374647

In [5]:
#Sfrutto l'opzione "parse_dates" della read.csv per trasformare in formato datetime le colonne di interesse
loans=dd.read_csv('/Users/caterinapisani/Progetto FCS/KIVA/loans.csv', parse_dates=['planned_expiration_time', 'disburse_time'])

In [14]:
#Creo un dataframe con solo le colonne di interesse per semplicità di visualizzazione
loans_time = loans[['loan_id','planned_expiration_time','disburse_time']]
loans_time.head()

,loan_id,planned_expiration_time,disburse_time
0,657307,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00
1,657259,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00
2,658010,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00
3,659347,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00
4,656933,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00


In [12]:
#Controllo che il tipo di dati sia effettivamente cambiato
loans_time.dtypes

loan_id                                  int64
planned_expiration_time    datetime64[ns, UTC]
disburse_time              datetime64[ns, UTC]
dtype: object

In [19]:
#Ora posso creare una nuova colonna "duration" data dalla differenza tra planned expiration time e disburse time

loans_duration = loans_time.assign(duration=loans['planned_expiration_time']-loans['disburse_time'])

In [20]:
loans_duration.head()

,loan_id,planned_expiration_time,disburse_time,duration
0,657307,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,53 days 19:30:06
1,657259,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,96 days 14:25:07
2,658010,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,37 days 13:10:05
3,659347,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,34 days 19:10:02
4,656933,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,57 days 22:10:02


In [21]:
loans_duration.tail()

,loan_id,planned_expiration_time,disburse_time,duration
27092,988180,2016-01-02 01:00:03+00:00,2015-11-23 08:00:00+00:00,39 days 17:00:03
27093,988213,2016-01-02 16:40:07+00:00,2015-11-24 08:00:00+00:00,39 days 08:40:07
27094,989109,2016-01-03 22:20:04+00:00,2015-11-13 08:00:00+00:00,51 days 14:20:04
27095,989143,2016-01-05 08:50:02+00:00,2015-11-03 08:00:00+00:00,63 days 00:50:02
27096,989240,2016-01-03 20:50:06+00:00,2015-11-03 08:00:00+00:00,61 days 12:50:06


In [25]:
#Voglio sapere il numero di valori mancanti nella colonna duration per poterli confrontare
#con Tot_Na trovato in precedenza 
Na_duration = loans.duration.isna().sum().compute()
Na_duration

374645

In [26]:
#La colonna duration presenta 2 valori mancanti in meno rispetto al Tot_Na 
#questo significa che in due casi "planned_expiration_time" e "disburse_time" presentano Na nella stessa riga
Tot_Na

374647

### Per lo svolgimento dei successivi punti del progetto, verrà adottata la libreria "mysql.connector"

In [27]:
import mysql.connector
from tabulate import tabulate


In [28]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="ProgettoFCS",
  database="kivapisani"
)

mycursor = mydb.cursor()

# 3. Find the lenders that have funded at least twice.

In [29]:
query_1 = """SELECT lend_name, count(status) as funded
FROM norm_loans_lenders LEFT JOIN loans ON loans.loan_id=norm_loans_lenders.loan_id
WHERE status='funded'
GROUP BY lend_name
HAVING funded>1
LIMIT 10"""

In [30]:
mycursor.execute(query_1)
myresult = mycursor.fetchall()
print(tabulate(myresult, headers=['Lend_name', 'Funded'], tablefmt='pretty'))

+--------------------+--------+
|     Lend_name      | Funded |
+--------------------+--------+
|   bernadette6835   |   27   |
|      jean2544      |  417   |
|        geko        |   33   |
|      lynn9800      |   4    |
|      brad4706      |  130   |
|       muc888       |  913   |
|     dougal1825     |  1141  |
| jensdamsgaardvanar |  7815  |
|     maeve9455      |   8    |
|      toni3909      |   34   |
+--------------------+--------+


# 4. For each country, compute how many loans have involved that country as borrowers.

In [31]:
query_2 = """SELECT country_name, count(loan_id) as count
FROM loans
GROUP BY country_name
ORDER BY country_name"""

In [32]:
mycursor.execute(query_2)
myresult = mycursor.fetchall()
print(tabulate(myresult, headers=['Country_name', 'Count'], tablefmt='pretty'))

+--------------------------------------+--------+
|             Country_name             | Count  |
+--------------------------------------+--------+
|             Afghanistan              |  2337  |
|               Albania                |  3075  |
|               Armenia                | 13952  |
|              Azerbaijan              | 10172  |
|                Belize                |  218   |
|                Benin                 |  5946  |
|                Bhutan                |   2    |
|               Bolivia                | 25250  |
|        Bosnia and Herzegovina        |  608   |
|               Botswana               |   1    |
|                Brazil                |  482   |
|               Bulgaria               |  296   |
|             Burkina Faso             |  3489  |
|               Burundi                |  1727  |
|               Cambodia               | 79701  |
|               Cameroon               |  5085  |
|                Canada                |   1    |


# 5. For each country, compute the overall amount of money borrowed.

In [33]:
query_3 = """Select country_name, sum(loan_amount) as money_borrowed
from loans
group by country_name
order by country_name"""

In [34]:
mycursor.execute(query_3)
myresult = mycursor.fetchall()
print ((tabulate(myresult, headers=['Country_name', 'Money_borrowed'], tablefmt='pretty')))

+--------------------------------------+----------------+
|             Country_name             | Money_borrowed |
+--------------------------------------+----------------+
|             Afghanistan              |    1967950     |
|               Albania                |    4307350     |
|               Armenia                |    22950475    |
|              Azerbaijan              |    14784625    |
|                Belize                |     150175     |
|                Benin                 |    3865825     |
|                Bhutan                |     20000      |
|               Bolivia                |    44226725    |
|        Bosnia and Herzegovina        |     477250     |
|               Botswana               |      8000      |
|                Brazil                |    1192325     |
|               Bulgaria               |     375300     |
|             Burkina Faso             |    4085200     |
|               Burundi                |    5233450     |
|             

# 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [35]:
query_4 = """Select distinct country_name, 
sum(loan_amount),
sum(loan_amount)*100/(select sum(loan_amount) from loans) as perc_money_borrowed
from loans
group by country_name
order by country_name"""

In [36]:
mycursor.execute(query_4)
myresult = mycursor.fetchall()
print ((tabulate(myresult, headers=['Country_name','Money_borrowed', 'Perc_money_borrowed'], tablefmt='pretty')))

+--------------------------------------+----------------+---------------------+
|             Country_name             | Money_borrowed | Perc_money_borrowed |
+--------------------------------------+----------------+---------------------+
|             Afghanistan              |    1967950     |       0.1666        |
|               Albania                |    4307350     |       0.3646        |
|               Armenia                |    22950475    |       1.9426        |
|              Azerbaijan              |    14784625    |       1.2514        |
|                Belize                |     150175     |       0.0127        |
|                Benin                 |    3865825     |       0.3272        |
|                Bhutan                |     20000      |       0.0017        |
|               Bolivia                |    44226725    |       3.7435        |
|        Bosnia and Herzegovina        |     477250     |       0.0404        |
|               Botswana               |

# 7. Like the three previous points, but split for each year (with respect to disburse time ).

In [22]:
#Nella clausola where richiedo di estrarre solo le righe in cui disburse_time sia maggiore di planned_expiration_time
#Escludo anche le percentuali pari a 0
query_5 = """Select country_name, 
EXTRACT(YEAR from disburse_time) AS year, 
sum(loan_amount) as money_borrowed,
sum(loan_amount)/(select sum(loan_amount) from loans) as perc_money_borrowed
from loans
where disburse_time > planned_expiration_time
group by country_name, year
having perc_money_borrowed > 0.00009
ORDER BY country_name
LIMIT 50"""

In [38]:
mycursor.execute(query_5)
myresult = mycursor.fetchall()
print ((tabulate(myresult, headers=['Country_name', 'Disburse_time', 'Money_borrowed', 'Perc_money_borrowed'], tablefmt='pretty')))

+------------------------+---------------+----------------+---------------------+
|      Country_name      | Disburse_time | Money_borrowed | Perc_money_borrowed |
+------------------------+---------------+----------------+---------------------+
|      Afghanistan       |     2007      |     194975     |       0.0002        |
|      Afghanistan       |     2008      |     365375     |       0.0003        |
|      Afghanistan       |     2009      |     585125     |       0.0005        |
|      Afghanistan       |     2010      |     563350     |       0.0005        |
|      Afghanistan       |     2011      |     245125     |       0.0002        |
|        Armenia         |     2010      |     731875     |       0.0006        |
|        Armenia         |     2011      |    1546775     |       0.0013        |
|       Azerbaijan       |     2007      |    1892775     |       0.0016        |
|       Azerbaijan       |     2008      |    1395475     |       0.0012        |
|       Azerbaij

# 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


###### Da questo punto e fino alla fine del progetto smetto di usare Mysql connector, le query richieste dai punti successive risulterebbero troppo onerose computazionalmente e si perderebbe il vantaggio avuto nei punti precedenti. 

###### Inoltre, come primo passo per la risoluzione di questo punto, converto i dataframe Dask utilizzati nei primi 2 punti (norm_loans_lenders e loans). Questo perchè incontrerebbero dei limiti con le funzioni di join e group by che la richiesta prevede. Infatti, anche se molte delle operazioni di pandas possono essere replicate con Dask (anche più efficientemente), questo trova difficoltà  nel gestire operazioni come group-by, set_index e join su colonne rendendo il tutto più lento e dispendioso. 

In [6]:
#Converto i dataframe dask in pandas con la funzione .compute()
norm_loans_lenders = norm_loans_lenders.compute()

In [7]:
loans = loans.compute()

In [8]:
# comincio a ricavare il numero di lenders per ogni loan_id ed eseguo una group by per loan_id
lenders_per_loan = norm_loans_lenders.groupby("loan_id").count().reset_index().rename(columns={"lenders" : "lenders_count"})

In [9]:
lenders_per_loan.head()

,loan_id,lenders_count
0,84,3
1,85,2
2,86,3
3,88,3
4,89,4


In [10]:
# eseguo una merge con loans per avere anche le informazioni riguardanti il loan_amount
lenders_loan_amount = pd.merge(lenders_per_loan, loans, on="loan_id")[["loan_id", "lenders_count", "loan_amount"]]
#loans_ = loans.set_index('loan_id')
#lenders_per_loan_= lenders_per_loan.set_index('loan_id')
#lenders_loan_amount= lenders_per_loan_.merge(loans_, left_index=True, right_index=True)

KeyboardInterrupt: 

In [ ]:
#Aggiungo una colonna in cui calcolo il loan_amount per lender, assumendo che tutti abbiano contribuito con lo stesso ammontare

lenders_loan_amount["amount_per_person"] = lenders_loan_amount["loan_amount"] / lenders_loan_amount["lenders_count"]

In [12]:
lenders_loan_amount.head()

,loan_id,lenders_count,loan_amount,amount_per_person
0,84,3,500.0,166.666667
1,85,2,500.0,250.000000
2,86,3,500.0,166.666667
3,88,3,300.0,100.000000
4,89,4,500.0,125.000000


In [13]:
#Ora unisco norm_loans_lenders e lenders_loan_amount in modo da avere un df con "amount_per_person" associato ad ogni lender

loans_lenders_merged = pd.merge(norm_loans_lenders, lenders_loan_amount, on="loan_id", how="left")

In [15]:
#Ora raggruppo per lender e sommo, ottenendo il totale prestato da ogni lender.

lenders_overall_lent = loans_lenders_merged.groupby("lenders")["amount_per_person"].sum().to_frame().reset_index()

In [16]:
lenders_overall_lent.head()

,lenders,amount_per_person
0,000,1672.618411
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117


# 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


In [69]:
lenders = pd.read_csv('/Users/caterinapisani/Progetto FCS/KIVA/lenders.csv')

In [72]:
lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0


In [13]:
lenders.isna().sum()

permanent_name             0
display_name            2768
city                 1619306
state                1713481
country_code         1458635
member_since               0
occupation           1844514
loan_because         2174852
loan_purchase_num     894281
invited_by           1852349
num_invited                0
dtype: int64

###### Come riportato nel testo della richiesta, vi sono molti valori mancanti nella colonna "country_code" e scegliamo di assumere che la distribuzione tra i Paesi dai valori mancanti sia la stessa di quelli in cui compare un valore. Dunque, il primo problema di cui occuparsi è quello del calcolo della distribuzione.  

In [78]:
#estraggo i country code non nulli e creo "lenders_notnull"
lenders_notnull = lenders.loc[lenders["country_code"].notnull()].reset_index()
lenders_notnull

,index,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,16,naresh2074,Naresh,NaN,NaN,US,1461303119,NaN,NaN,3.0,NaN,0
1,31,christina27976796,Christina,NaN,NaN,US,1461303341,NaN,NaN,1.0,Peter Tan,0
2,37,vikas1098,Vikas,Bengaluru,NaN,IN,1461301846,Software Engineer,NaN,2.0,NaN,0
3,39,qian1385,Qian,NaN,NaN,US,1461302036,NaN,NaN,2.0,NaN,0
4,42,xigg8769,Darla,Madison,NaN,US,1461302503,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
890534,2349158,rakhi,Rakhi,New York,New York,US,1342100607,Student,I care.,4.0,NaN,0
890535,2349159,vicki5374,Vicki,Austin,Texas,US,1342101743,NaN,NaN,1.0,Vicki,1
890536,2349161,jennifer5879,Jennifer,Riverview,New Brunswick,CA,1342092207,NaN,NaN,1.0,Beverly,1
890537,2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0


In [81]:
lenders_notnull=lenders_notnull[['index','country_code']]
lenders_notnull

,index,country_code
0,16,US
1,31,US
2,37,IN
3,39,US
4,42,US
...,...,...
890534,2349158,US
890535,2349159,US
890536,2349161,CA
890537,2349171,US


In [82]:
tot_notnull_lenders = len(lenders_notnull.index)
tot_notnull_lenders

890539

In [83]:
#estraggo i country code nulli e creo "lenders_null"
lenders_null = lenders.loc[lenders["country_code"].isnull()].reset_index()

In [84]:
lenders_null = lenders_null[['index','country_code']]
lenders_null

,index,country_code
0,0,NaN
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN
...,...,...
1458630,2349167,NaN
1458631,2349168,NaN
1458632,2349169,NaN
1458633,2349170,NaN


In [ ]:
#Calcolo la distribuzione delle nazioni nel dataframe lenders_notnull: innanzitutto eseguo una group_by 
#e una count sul country_code per ottenere il num di lender country 

lenders_per_country = lenders_notnull.groupby("country_code").count().reset_index().rename(columns = {"index":"n_lenders"})
lenders_per_country

In [ ]:
#secondariamente calcolo la percentuale del num di lenders per country sul totale dei lenders non nulli

lenders_per_country["percentage"] = users_per_country["n_lenders"]/tot_notnull_lenders*100

lenders_per_country

In [ ]:
somma_perc = lenders_per_country["percentage"].sum()
somma_perc

In [ ]:
#Infine, normalizzo tutte le percentuali dividendole per la loro somma in modo che il totale sia pari a 1 e non si ottengano errori

lenders_per_country["percentage"] /= lenders_per_country["percentage"].sum()
somma_perc = lenders_per_country["percentage"].sum()
somma_perc

In [30]:
#Ora posso riempire il dataframe lenders_null in modo che abbia la stessa distribuzione di lenders_notnull.
#Per fare questo uso la funzione np.random.choice (inserendo come seed '2208', orario corrente)
#In questo modo assegno la nazionalità in modo randomico e non sistematico (x. es. partendo dall'alto), ottenendo la stessa distribuzione del df non nullo

np.random.seed(2208)
lenders_null["country_code"] = np.random.choice(lenders_per_country["country_code"], size=len(lenders_null.index), p = lenders_per_country["percentage"])

lenders_null

,index,country_code
0,0,GB
1,1,US
2,2,US
3,3,US
4,4,US
...,...,...
1458630,2349167,US
1458631,2349168,NO
1458632,2349169,US
1458633,2349170,IT


In [ ]:
#Inserisco nel df "lenders" originario i df lenders_notnull e lenders_null in cui non compaiono più country_code nulli
lenders = pd.concat([lenders_notnull, lenders_null]).drop(columns="index")

# 10. Which country has the highest ratio between the difference computed at the previous point and the population?

# 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

# 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.